In [1]:
time_alice = [
  [10 4];
  [20 5];
  [30 6];
  [40 7];
  [50 10];
  [60 8];
  [70 6];
  [80 2]       
]

time_bob = [
  [10 4];
  [20 6];
  [30 10];
  [40 20];
  [50 40];
  [60 11];
  [70 5];
  [80 4]
]

display(time_bob)
println(eltype(time_bob))
println(size(time_bob))

8×2 Array{Int64,2}:
 10   4
 20   6
 30  10
 40  20
 50  40
 60  11
 70   5
 80   4

Int64
(8, 2)


In [2]:
time_bob

8×2 Array{Int64,2}:
 10   4
 20   6
 30  10
 40  20
 50  40
 60  11
 70   5
 80   4

In [3]:
] status StatsBase

    Status `~/.julia/environments/v1.3/Project.toml`
  [864edb3b] DataStructures v0.17.11
  [2913bbd2] StatsBase v0.32.2
  [9a3f8284] Random 
  [2f01184e] SparseArrays 


In [21]:
# ] add StatsBase

LoadError: syntax: unexpected "]"

In [22]:
using StatsBase

In [6]:
sample(1:6, Weights(ones(6)), (100,100) ) # throw a dice 10000 times

100×100 Array{Int64,2}:
 4  3  4  4  6  3  2  4  6  1  4  6  5  …  1  3  3  2  5  3  3  2  4  6  3  1
 5  3  1  4  3  5  5  3  4  6  2  2  4     4  1  5  4  2  3  3  5  6  6  5  4
 5  2  5  4  3  2  2  4  3  2  3  2  5     5  5  6  3  3  4  4  2  2  5  4  5
 3  4  6  4  4  4  3  3  5  6  3  5  1     1  1  1  4  6  1  5  3  3  3  2  1
 4  2  4  4  6  3  5  2  2  3  1  2  2     1  2  2  5  6  6  2  3  4  5  2  6
 4  3  1  6  1  5  1  2  1  2  3  2  3  …  2  5  4  1  5  1  6  4  5  3  1  6
 6  1  5  2  5  5  5  3  1  6  5  6  1     6  2  1  5  2  1  6  4  2  4  4  1
 2  2  4  6  3  5  6  5  4  4  5  1  4     5  2  2  2  4  4  4  2  4  6  4  6
 5  5  4  3  6  3  2  6  5  5  5  3  4     5  3  2  4  3  3  5  3  4  6  2  2
 1  6  1  6  5  4  5  4  2  5  6  1  4     5  6  4  3  6  4  3  4  3  3  4  3
 1  2  2  1  2  6  3  1  5  3  2  5  1  …  4  6  5  3  4  1  5  4  1  2  3  2
 1  6  2  2  1  3  4  3  4  1  6  3  6     3  6  2  3  3  1  3  5  3  2  5  3
 6  2  5  4  5  5  5  6  4  2  6  5  6  

In [7]:
using Random
Random.seed!(0)

sample(time_bob[:,1], Weights(time_bob[:,2]),100)


100-element Array{Int64,1}:
 60
 70
 30
 30
 40
 40
 20
 20
 40
 80
 50
 50
 40
  ⋮
 50
 50
 50
 20
 50
 40
 80
 40
 60
 20
 50
 60

In [8]:
using Parameters, DataStructures

In [9]:
#] add Parameters DataStructures

In [10]:
using DataStructures

q = PriorityQueue{String, Tuple{Int,Int}}()
enqueue!(q, "event1", (100,10))
enqueue!(q, "event2", (50,20))
enqueue!(q, "event3", (200,5))
enqueue!(q, "event4", (200,4))

display(q)
display(dequeue_pair!(q))
display(dequeue_pair!(q))
display(dequeue_pair!(q))
display(dequeue_pair!(q))

display(q)

PriorityQueue{String,Tuple{Int64,Int64},Base.Order.ForwardOrdering} with 4 entries:
  "event2" => (50, 20)
  "event1" => (100, 10)
  "event4" => (200, 4)
  "event3" => (200, 5)

"event2" => (50, 20)

"event1" => (100, 10)

"event4" => (200, 4)

"event3" => (200, 5)

PriorityQueue{String,Tuple{Int64,Int64},Base.Order.ForwardOrdering} with 0 entries

In [11]:
struct Event
    description::String
    what::Symbol
end

In [12]:
using Parameters
@with_kw mutable struct Point2
    x::Int
    y::Int
    z::Int = 0
end

p = Point2(x=3,y=4)
p

p.x = 7

p

Point2
  x: Int64 7
  y: Int64 4
  z: Int64 0


In [13]:
using Parameters
@with_kw mutable struct ProductionLine
    tick::Int  = 0       # simulation step
    clock::Int = 0       # time in simulated world in seconda
    alice_has_box_for_bob::Bool = false
    bob_glues_box::Bool = false
    completed_boxes::Int = 0 # completed boxes
    time_alice::Matrix{Int}
    time_bob::Matrix{Int}
    q::PriorityQueue{Event, Tuple{Int,Int}}=PriorityQueue{Event, Tuple{Int,Int}}()
end    

ProductionLine

In [14]:
a,b= (4,6)
a+b

10

In [15]:
function step(sim::ProductionLine; log = true)
    length(sim.q) == 0 && enqueue!(sim.q, Event("Alice takes the box",:alice_take), (sim.clock,0)) 
    message_add = ""
    event, time_priority = dequeue_pair!(sim.q)    
    sim.clock = time_priority[1]
    if event.what == :alice_take
        enqueue!(sim.q, Event("Alice completed box",:alice_assemble), 
            (sim.clock +sample(sim.time_alice[:,1], Weights(sim.time_alice[:,2])), 1) ) 
    elseif event.what == :alice_assemble
        sim.alice_has_box_for_bob = true
        enqueue!(sim.q, Event("Bob checks if he take the box",:bob_check), (sim.clock,2))
    elseif event.what == :bob_check
        if !sim.bob_glues_box && sim.alice_has_box_for_bob
            sim.bob_glues_box = true
            sim.alice_has_box_for_bob = false
            message_add = " and he took it from Alice!"
            enqueue!(sim.q, Event("Alice takes the box",:alice_take), (sim.clock,3)) 
            enqueue!(sim.q, Event("Bob glued the box",:bob_glue), 
                (sim.clock +sample(sim.time_bob[:,1], Weights(sim.time_bob[:,2])), 4) )             
        else 
            if sim.bob_glues_box 
                message_add *=  "[he is still working on the previous one] "
            end
            if ! sim.alice_has_box_for_bob 
                message_add *=  "[Alice is not ready yet ] "
            end
        end
    elseif event.what == :bob_glue
        sim.completed_boxes += 1
        sim.bob_glues_box = false
        enqueue!(sim.q, Event("Bob checks if he can take the box",:bob_check), (sim.clock, 5))
    else
        println("Uknown event? $(event.what)")
    end   
    sim.tick += 1
    log && println("tick=$(sim.tick): $(event.description) $message_add  clock=$(sim.clock), completed=$(sim.completed_boxes)")
    
end

step (generic function with 1 method)

In [16]:
sim = ProductionLine(time_alice=time_alice, time_bob=time_bob)
while sim.clock <= 3600*8
    step(sim; log=false)
    #flush(stdout)
    #display(sim.q)
    #flush(stdout)
end
display(sim)

ProductionLine
  tick: Int64 2625
  clock: Int64 28840
  alice_has_box_for_bob: Bool false
  bob_glues_box: Bool false
  completed_boxes: Int64 525
  time_alice: Array{Int64}((8, 2)) [10 4; 20 5; … ; 70 6; 80 2]
  time_bob: Array{Int64}((8, 2)) [10 4; 20 6; … ; 70 5; 80 4]
  q: PriorityQueue{Event,Tuple{Int64,Int64},Base.Order.ForwardOrdering}


In [17]:
@with_kw mutable struct ProductionLineStorage
    tick::Int  = 0       # simulation step
    clock::Int = 0       # time in simulated world in seconda
    storage::Int = 0
    alice_has_box_for_bob::Bool = false
    bob_glues_box::Bool = false
    completed_boxes::Int = 0 # completed boxes
    time_alice::Matrix{Int}
    time_bob::Matrix{Int}
    q::PriorityQueue{Event, Tuple{Int,Int}}=PriorityQueue{Event, Tuple{Int,Int}}()
end  

ProductionLineStorage

In [18]:
function step(sim::ProductionLineStorage; log = true)
    length(sim.q) == 0 && enqueue!(sim.q, Event("Alice takes the box",:alice_take), (sim.clock,0)) 
    message_add = ""
    event, time_priority = dequeue_pair!(sim.q)    
    sim.clock = time_priority[1]
    if event.what == :alice_take
        enqueue!(sim.q, Event("Alice completed box",:alice_assemble), 
            (sim.clock +sample(sim.time_alice[:,1], Weights(sim.time_alice[:,2])), 1) ) 
        
    elseif event.what == :alice_assemble
        sim.storage += 1
        enqueue!(sim.q, Event("Bob checks if he take the box",:bob_check), (sim.clock,2))
        enqueue!(sim.q, Event("Alice takes the box",:alice_take), (sim.clock,3))        
    elseif event.what == :bob_check
        if !sim.bob_glues_box && sim.storage > 0
            sim.bob_glues_box = true
            sim.storage -= 1
            message_add = " and he took it from Storage!"             
            enqueue!(sim.q, Event("Bob glued the box",:bob_glue), 
                (sim.clock +sample(sim.time_bob[:,1], Weights(sim.time_bob[:,2])), 4) )             
        else 
            if sim.bob_glues_box 
                message_add *=  "[he is still working on the previous one] "
            end
            if sim.storage == 0
                message_add *=  "[No boxes in storage ] "
            end
        end
    elseif event.what == :bob_glue
        sim.completed_boxes += 1
        sim.bob_glues_box = false
        enqueue!(sim.q, Event("Bob checks if he can take the box",:bob_check), (sim.clock, 5))
    else
        println("Uknown event? $(event.what)")
    end   
    sim.tick += 1
    log && println("tick=$(sim.tick): $(event.description) $message_add  clock=$(sim.clock), S=$(sim.storage) completed=$(sim.completed_boxes)")
    
end

step (generic function with 2 methods)

In [19]:
methods(step)

# 2 methods for generic function "step":
[1] step(sim::ProductionLineStorage; log) in Main at In[18]:2
[2] step(sim::ProductionLine; log) in Main at In[15]:2

In [20]:
sim = ProductionLineStorage(time_alice=time_alice, time_bob=time_bob)
s_history = Int[]
while sim.clock <= 3600*8
    step(sim; log=false)
    push!(s_history,sim.storage)
    #flush(stdout)
    #display(sim.q)
    #flush(stdout)
end
display(sim)



ProductionLineStorage
  tick: Int64 3167
  clock: Int64 28840
  storage: Int64 15
  alice_has_box_for_bob: Bool false
  bob_glues_box: Bool true
  completed_boxes: Int64 624
  time_alice: Array{Int64}((8, 2)) [10 4; 20 5; … ; 70 6; 80 2]
  time_bob: Array{Int64}((8, 2)) [10 4; 20 6; … ; 70 5; 80 4]
  q: PriorityQueue{Event,Tuple{Int64,Int64},Base.Order.ForwardOrdering}


## Homework:

Make plots (for 1 simulation):
- utilization (4 bars)
    of Alice time with and witout Storage (what is percentage of time that Alice is waitng for Bob)
    of Bob's time with and witout Storage (what is percentage of time that Bob is waitng for Alice)
- storage usage over time 

Use either Python or Julia

Provide code + 2 pictures (png format)